<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 636 kB 43.2 MB/s 
     |████████████████████████████████| 3.3 MB 16.4 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 


In [ ]:
import torch
from transformers import BertModel, BertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

<b>2. Load the vocabulary and ProtBert Model</b>

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

In [ ]:
model = BertModel.from_pretrained("Rostlab/prot_bert")

Downloading:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
device

device(type='cuda', index=0)

In [ ]:
model = model.to(device)
model = model.eval()

In [ ]:

dataset = pd.read_csv(r"/content/drive/MyDrive/new_AntiMicrobial_and_NonAntiMicrobial_Peptides.csv")
print(len(dataset))
dataset.head(3)

13045


,seq,class
0,M A Q E F V N C K I Q S G K V V V F I K P T C ...,non_Anti_microbial
1,M E F F N I L Q S A C N D V N L D F N D K K Y ...,non_Anti_microbial
2,M A P K I A I V Y Y S M Y G H I K K M A D A E ...,non_Anti_microbial


In [1]:
#num_1 = 13000
#num_2 = len(dataset)
#dataset1 = dataset.iloc[num_1:num_2]
#print(len(dataset1))
#dataset1.head(3)

In [ ]:
#dataset1.tail(3)

,seq,class
13042,M P T I S Q L I R K G R K T V A S K S T A P A ...,non_Anti_microbial
13043,D C Y E D W S R C T P G T S F L T G I L W K D ...,Anti_microbial
13044,S L Q P G A P N F P M P G S Q L P T S I T S N ...,Anti_microbial


In [ ]:
sequences_Example = list(dataset1["seq"])
len(sequences_Example)

45

In [ ]:
#sequences_Example = ["A A C S D R A H G H I C E S F K S F C K D S G R N G V K L R A N C K K T C G L C ",
#                     "A A E F P D F Y D S E E Q M G P H Q E A E D E K D R A D Q R V L T E E E K K E L E N L A A M D L E L Q K I A E K F S Q R "]

In [ ]:



#sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

In [ ]:
#sequences_Example


<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [ ]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [ ]:
len(input_ids[0])

257

In [2]:
#attention_mask

<b>6. Extracting sequences' features and load it into the CPU if needed<b>

In [ ]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [ ]:
embedding = embedding.cpu().numpy()

In [ ]:

all_vectors = len(embedding)
all_vectors

45

In [ ]:
dictt = dict()
for point_num in range(len(embedding)):
    cls_token_vector = list(embedding[point_num][0])
    dictt[point_num+num_1] = cls_token_vector

In [ ]:
np.save('vectors_dict_'+str(num_1)+"_"+str(num_2)+'.npy', dictt)
#read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()

In [ ]:
!cp "/content/..npy" "/content/drive/MyDrive/Bayes labs"

In [ ]:
read_dictionary = np.load('vectors_dict_2000_2500.npy',allow_pickle='TRUE').item()
read_dictionary.get(2001)

In [ ]:
np.save('bert_features.npy', bert_features)
#read_dictionary = np.load('my_file.npy',allow_pickle='TRUE').item()

In [ ]:
!cp "/content/bert_features.npy" "/content/drive/MyDrive/Bayes labs"

In [ ]:
array = bert_features.get(0)
range(len(bert_features.get(0)))

1024

In [ ]:
import pandas as pd
columns_list = [(str("Feature")+str(num+1)) for num in range(len(bert_features.get(0)))]
bert_dataframe = pd.DataFrame()

for key in list(bert_features.keys()):
    bert_dataframe = bert_dataframe.append(pd.Series(bert_features.get(key), index=columns_list), ignore_index=True)
print(len(bert_dataframe))
bert_dataframe.head(3)

13045


,Feature1,Feature10,Feature100,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature101,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature102,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,Feature103,Feature104,Feature105,Feature106,Feature107,Feature108,Feature109,Feature11,Feature110,Feature111,...,Feature963,Feature964,Feature965,Feature966,Feature967,Feature968,Feature969,Feature97,Feature970,Feature971,Feature972,Feature973,Feature974,Feature975,Feature976,Feature977,Feature978,Feature979,Feature98,Feature980,Feature981,Feature982,Feature983,Feature984,Feature985,Feature986,Feature987,Feature988,Feature989,Feature99,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999
0,0.069630,-0.086695,0.013663,0.027432,0.134993,0.020873,-0.064566,-0.092742,-0.031703,-0.062025,-0.001946,0.072060,-0.021253,0.075823,-0.000871,-0.091445,0.071739,-0.012103,0.013119,-0.053159,0.060784,-0.099561,-0.142787,0.036709,-0.118996,-0.097751,-0.061995,-0.051084,-0.038646,-0.020350,0.000058,0.114891,0.066714,-0.030405,-0.044341,-0.065679,-0.019279,-0.038282,0.115485,0.063072,...,0.081762,-0.069268,-0.042372,-0.040319,-0.040733,-0.017331,-0.038666,-0.018984,0.162583,0.044741,-0.106462,-0.014764,-0.054419,0.094815,0.006867,0.059820,-0.024422,0.022515,-0.109301,-0.023033,-0.087442,0.089245,-0.132111,-0.026248,0.058767,0.043572,-0.095108,-0.108759,0.024416,0.065246,-0.089847,-0.015661,-0.107151,-0.006303,0.003388,0.025380,0.001922,-0.024166,-0.007612,-0.077760
1,0.158570,-0.145537,0.050485,0.054790,0.119621,-0.117958,-0.008106,-0.071025,0.019424,-0.026795,-0.065976,-0.045075,-0.033450,-0.015239,0.004089,-0.091568,0.090431,0.037361,0.209537,-0.030037,-0.083185,-0.073892,-0.107537,0.053009,-0.110557,-0.203266,-0.116658,-0.052270,-0.067016,0.067302,-0.112086,0.121462,0.079947,-0.171350,-0.124621,-0.109205,0.008954,0.003898,0.083383,-0.047253,...,0.077464,-0.193687,-0.062979,-0.046658,-0.052407,-0.038375,0.072362,0.233282,0.102295,-0.062372,-0.136478,-0.161449,0.065954,0.108111,0.016315,0.100653,-0.031013,0.057518,-0.042499,-0.089930,-0.198744,0.132785,0.042520,-0.080131,0.071113,0.005787,-0.082780,-0.009048,0.046565,0.192974,-0.040207,-0.089750,-0.103536,0.090986,-0.072441,0.074443,0.065187,-0.018859,0.063093,-0.122281
2,0.111323,-0.111279,0.029424,0.038788,0.063038,-0.065605,-0.035414,-0.062052,-0.000771,-0.018625,-0.052999,0.018849,-0.025234,0.073426,-0.004726,-0.055147,0.102750,-0.021590,0.067670,-0.016135,0.022117,-0.088876,-0.142062,0.005176,-0.118729,-0.166395,-0.042158,-0.090205,-0.038277,0.002191,-0.054010,0.089787,0.048523,-0.066153,-0.100489,-0.081120,-0.002940,-0.036637,0.111943,0.019696,...,0.074795,-0.110044,-0.042067,-0.031159,-0.040284,0.009608,0.008083,0.057910,0.111550,-0.026540,-0.087033,-0.047687,-0.019794,0.067864,0.062954,0.079960,-0.029034,0.055310,-0.104703,-0.015723,-0.096233,0.063445,-0.067721,-0.051715,0.048370,0.046643,-0.073014,-0.068133,0.090216,0.112194,-0.050809,-0.036625,-0.086690,0.051850,0.001070,0.044721,-0.008755,-0.021717,0.047530,-0.103937


In [ ]:
bert_dataframe.columns

Index(['Feature1', 'Feature10', 'Feature100', 'Feature1000', 'Feature1001',
       'Feature1002', 'Feature1003', 'Feature1004', 'Feature1005',
       'Feature1006',
       ...
       'Feature990', 'Feature991', 'Feature992', 'Feature993', 'Feature994',
       'Feature995', 'Feature996', 'Feature997', 'Feature998', 'Feature999'],
      dtype='object', length=1024)

In [ ]:
len(bert_dataframe.columns)

1024

In [ ]:
final_bert_dataframe = pd.DataFrame()
for i in range(len(bert_dataframe.columns)):
    column = list(bert_dataframe["Feature"+str(i+1)])
    final_bert_dataframe["Feature"+str(i+1)] = column
print(len(final_bert_dataframe))
final_bert_dataframe.head(3)

13045


,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature985,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024
0,0.069630,0.037715,0.094908,-0.204495,0.216357,-0.067500,-0.129693,0.017480,0.021069,-0.086695,-0.038282,-0.004839,0.094959,0.033578,-0.061521,0.021535,-0.077589,0.088890,0.063067,-0.089765,-0.006309,0.040383,-0.001253,-0.167778,-0.032887,0.031695,0.069038,0.002625,0.009161,-0.091343,-0.081996,-0.053690,-0.129325,0.088868,0.090851,-0.073844,0.145810,-0.100443,-0.070930,-0.011890,...,0.058767,0.043572,-0.095108,-0.108759,0.024416,-0.089847,-0.015661,-0.107151,-0.006303,0.003388,0.025380,0.001922,-0.024166,-0.007612,-0.077760,0.027432,0.134993,0.020873,-0.064566,-0.092742,-0.031703,-0.062025,-0.001946,0.072060,-0.021253,-0.000871,-0.091445,0.071739,-0.012103,0.013119,-0.053159,0.060784,-0.099561,-0.142787,0.036709,-0.097751,-0.061995,-0.051084,-0.038646,-0.020350
1,0.158570,0.027016,0.084353,-0.261657,0.255672,-0.028979,-0.060672,0.003051,-0.072140,-0.145537,0.003898,0.022706,0.045013,0.127011,0.063842,-0.138697,-0.048299,0.098664,-0.026465,-0.073534,0.085967,0.080135,-0.108955,-0.152588,0.037963,-0.013081,0.093290,0.013057,0.058086,-0.135192,0.000354,0.136811,-0.193744,0.033134,0.083954,-0.091634,0.068099,0.003499,-0.105327,-0.154782,...,0.071113,0.005787,-0.082780,-0.009048,0.046565,-0.040207,-0.089750,-0.103536,0.090986,-0.072441,0.074443,0.065187,-0.018859,0.063093,-0.122281,0.054790,0.119621,-0.117958,-0.008106,-0.071025,0.019424,-0.026795,-0.065976,-0.045075,-0.033450,0.004089,-0.091568,0.090431,0.037361,0.209537,-0.030037,-0.083185,-0.073892,-0.107537,0.053009,-0.203266,-0.116658,-0.052270,-0.067016,0.067302
2,0.111323,0.024840,0.122577,-0.200168,0.211283,-0.039343,-0.109409,0.027931,-0.040814,-0.111279,-0.036637,-0.010607,0.046858,0.062787,-0.046101,-0.027495,-0.069926,0.055665,0.011457,-0.077800,0.052512,0.011069,-0.041876,-0.195665,-0.018367,0.050151,0.093020,0.008231,0.008631,-0.097777,-0.039131,0.053937,-0.143924,0.058895,0.053202,-0.087114,0.087996,-0.057728,-0.081531,-0.049334,...,0.048370,0.046643,-0.073014,-0.068133,0.090216,-0.050809,-0.036625,-0.086690,0.051850,0.001070,0.044721,-0.008755,-0.021717,0.047530,-0.103937,0.038788,0.063038,-0.065605,-0.035414,-0.062052,-0.000771,-0.018625,-0.052999,0.018849,-0.025234,-0.004726,-0.055147,0.102750,-0.021590,0.067670,-0.016135,0.022117,-0.088876,-0.142062,0.005176,-0.166395,-0.042158,-0.090205,-0.038277,0.002191


In [ ]:
final_bert_dataframe.to_csv()

In [ ]:
final_bert_dataframe.to_csv("bert_features.csv", encoding='utf-8', index=False)

In [ ]:
!cp "bert_features.csv" "/content/drive/MyDrive/Bayes labs"

In [ ]:
classes = dataset["class"]

In [ ]:
final_bert_dataframe["labels"] = classes

In [ ]:
final_bert_dataframe.to_csv("bert_dataframe.csv", encoding='utf-8', index=False)

In [ ]:
!cp "bert_dataframe.csv" "/content/drive/MyDrive/Bayes labs"

In [ ]:
data = pd.read_csv("bert_dataframe.csv")
print(len(bert_dataframe))
bert_dataframe.head(3)

13045


,Feature1,Feature10,Feature100,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature101,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature102,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,Feature103,Feature104,Feature105,Feature106,Feature107,Feature108,Feature109,Feature11,Feature110,Feature111,...,Feature963,Feature964,Feature965,Feature966,Feature967,Feature968,Feature969,Feature97,Feature970,Feature971,Feature972,Feature973,Feature974,Feature975,Feature976,Feature977,Feature978,Feature979,Feature98,Feature980,Feature981,Feature982,Feature983,Feature984,Feature985,Feature986,Feature987,Feature988,Feature989,Feature99,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999
0,0.069630,-0.086695,0.013663,0.027432,0.134993,0.020873,-0.064566,-0.092742,-0.031703,-0.062025,-0.001946,0.072060,-0.021253,0.075823,-0.000871,-0.091445,0.071739,-0.012103,0.013119,-0.053159,0.060784,-0.099561,-0.142787,0.036709,-0.118996,-0.097751,-0.061995,-0.051084,-0.038646,-0.020350,0.000058,0.114891,0.066714,-0.030405,-0.044341,-0.065679,-0.019279,-0.038282,0.115485,0.063072,...,0.081762,-0.069268,-0.042372,-0.040319,-0.040733,-0.017331,-0.038666,-0.018984,0.162583,0.044741,-0.106462,-0.014764,-0.054419,0.094815,0.006867,0.059820,-0.024422,0.022515,-0.109301,-0.023033,-0.087442,0.089245,-0.132111,-0.026248,0.058767,0.043572,-0.095108,-0.108759,0.024416,0.065246,-0.089847,-0.015661,-0.107151,-0.006303,0.003388,0.025380,0.001922,-0.024166,-0.007612,-0.077760
1,0.158570,-0.145537,0.050485,0.054790,0.119621,-0.117958,-0.008106,-0.071025,0.019424,-0.026795,-0.065976,-0.045075,-0.033450,-0.015239,0.004089,-0.091568,0.090431,0.037361,0.209537,-0.030037,-0.083185,-0.073892,-0.107537,0.053009,-0.110557,-0.203266,-0.116658,-0.052270,-0.067016,0.067302,-0.112086,0.121462,0.079947,-0.171350,-0.124621,-0.109205,0.008954,0.003898,0.083383,-0.047253,...,0.077464,-0.193687,-0.062979,-0.046658,-0.052407,-0.038375,0.072362,0.233282,0.102295,-0.062372,-0.136478,-0.161449,0.065954,0.108111,0.016315,0.100653,-0.031013,0.057518,-0.042499,-0.089930,-0.198744,0.132785,0.042520,-0.080131,0.071113,0.005787,-0.082780,-0.009048,0.046565,0.192974,-0.040207,-0.089750,-0.103536,0.090986,-0.072441,0.074443,0.065187,-0.018859,0.063093,-0.122281
2,0.111323,-0.111279,0.029424,0.038788,0.063038,-0.065605,-0.035414,-0.062052,-0.000771,-0.018625,-0.052999,0.018849,-0.025234,0.073426,-0.004726,-0.055147,0.102750,-0.021590,0.067670,-0.016135,0.022117,-0.088876,-0.142062,0.005176,-0.118729,-0.166395,-0.042158,-0.090205,-0.038277,0.002191,-0.054010,0.089787,0.048523,-0.066153,-0.100489,-0.081120,-0.002940,-0.036637,0.111943,0.019696,...,0.074795,-0.110044,-0.042067,-0.031159,-0.040284,0.009608,0.008083,0.057910,0.111550,-0.026540,-0.087033,-0.047687,-0.019794,0.067864,0.062954,0.079960,-0.029034,0.055310,-0.104703,-0.015723,-0.096233,0.063445,-0.067721,-0.051715,0.048370,0.046643,-0.073014,-0.068133,0.090216,0.112194,-0.050809,-0.036625,-0.086690,0.051850,0.001070,0.044721,-0.008755,-0.021717,0.047530,-0.103937


In [ ]:
bert_dataframe["la"]

In [ ]:
final_bert_dataframe.to_csv("dataframe.csv", encoding='utf-8', index=False)

In [ ]:
!cp "dataframe.csv" "/content/drive/MyDrive/Bayes labs"

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Bayes labs/dataframe.csv")
data.head(3)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,labels
0,0.069630,0.037715,0.094908,-0.204495,0.216357,-0.067500,-0.129693,0.017480,0.021069,-0.086695,-0.038282,-0.004839,0.094959,0.033578,-0.061521,0.021535,-0.077589,0.088890,0.063067,-0.089765,-0.006309,0.040383,-0.001253,-0.167778,-0.032887,0.031695,0.069038,0.002625,0.009161,-0.091343,-0.081996,-0.053690,-0.129325,0.088868,0.090851,-0.073844,0.145810,-0.100443,-0.070930,-0.011890,...,0.043572,-0.095108,-0.108759,0.024416,-0.089847,-0.015661,-0.107151,-0.006303,0.003388,0.025380,0.001922,-0.024166,-0.007612,-0.077760,0.027432,0.134993,0.020873,-0.064566,-0.092742,-0.031703,-0.062025,-0.001946,0.072060,-0.021253,-0.000871,-0.091445,0.071739,-0.012103,0.013119,-0.053159,0.060784,-0.099561,-0.142787,0.036709,-0.097751,-0.061995,-0.051084,-0.038646,-0.020350,non_Anti_microbial
1,0.158570,0.027016,0.084353,-0.261657,0.255672,-0.028979,-0.060672,0.003051,-0.072140,-0.145537,0.003898,0.022706,0.045013,0.127011,0.063842,-0.138697,-0.048299,0.098664,-0.026465,-0.073534,0.085967,0.080135,-0.108955,-0.152588,0.037963,-0.013081,0.093290,0.013057,0.058086,-0.135192,0.000354,0.136811,-0.193744,0.033134,0.083954,-0.091634,0.068099,0.003499,-0.105327,-0.154782,...,0.005787,-0.082780,-0.009048,0.046565,-0.040207,-0.089750,-0.103536,0.090986,-0.072441,0.074443,0.065187,-0.018859,0.063093,-0.122281,0.054790,0.119621,-0.117958,-0.008106,-0.071025,0.019424,-0.026795,-0.065976,-0.045075,-0.033450,0.004089,-0.091568,0.090431,0.037361,0.209537,-0.030037,-0.083185,-0.073892,-0.107537,0.053009,-0.203266,-0.116658,-0.052270,-0.067016,0.067302,non_Anti_microbial
2,0.111323,0.024840,0.122577,-0.200168,0.211283,-0.039343,-0.109409,0.027931,-0.040814,-0.111279,-0.036637,-0.010607,0.046858,0.062787,-0.046101,-0.027495,-0.069926,0.055665,0.011457,-0.077800,0.052512,0.011069,-0.041876,-0.195665,-0.018367,0.050151,0.093020,0.008231,0.008631,-0.097777,-0.039131,0.053937,-0.143924,0.058895,0.053202,-0.087114,0.087996,-0.057728,-0.081531,-0.049334,...,0.046643,-0.073014,-0.068133,0.090216,-0.050809,-0.036625,-0.086690,0.051850,0.001070,0.044721,-0.008755,-0.021717,0.047530,-0.103937,0.038788,0.063038,-0.065605,-0.035414,-0.062052,-0.000771,-0.018625,-0.052999,0.018849,-0.025234,-0.004726,-0.055147,0.102750,-0.021590,0.067670,-0.016135,0.022117,-0.088876,-0.142062,0.005176,-0.166395,-0.042158,-0.090205,-0.038277,0.002191,non_Anti_microbial


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/Bayes labs/dataframe.csv")
print("Total data points we have in our Dataset is :",len(data))
data.head(3)

Total data points we have in our Dataset is : 13045


,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,labels
0,0.069630,0.037715,0.094908,-0.204495,0.216357,-0.067500,-0.129693,0.017480,0.021069,-0.086695,-0.038282,-0.004839,0.094959,0.033578,-0.061521,0.021535,-0.077589,0.088890,0.063067,-0.089765,-0.006309,0.040383,-0.001253,-0.167778,-0.032887,0.031695,0.069038,0.002625,0.009161,-0.091343,-0.081996,-0.053690,-0.129325,0.088868,0.090851,-0.073844,0.145810,-0.100443,-0.070930,-0.011890,...,0.043572,-0.095108,-0.108759,0.024416,-0.089847,-0.015661,-0.107151,-0.006303,0.003388,0.025380,0.001922,-0.024166,-0.007612,-0.077760,0.027432,0.134993,0.020873,-0.064566,-0.092742,-0.031703,-0.062025,-0.001946,0.072060,-0.021253,-0.000871,-0.091445,0.071739,-0.012103,0.013119,-0.053159,0.060784,-0.099561,-0.142787,0.036709,-0.097751,-0.061995,-0.051084,-0.038646,-0.020350,non_Anti_microbial
1,0.158570,0.027016,0.084353,-0.261657,0.255672,-0.028979,-0.060672,0.003051,-0.072140,-0.145537,0.003898,0.022706,0.045013,0.127011,0.063842,-0.138697,-0.048299,0.098664,-0.026465,-0.073534,0.085967,0.080135,-0.108955,-0.152588,0.037963,-0.013081,0.093290,0.013057,0.058086,-0.135192,0.000354,0.136811,-0.193744,0.033134,0.083954,-0.091634,0.068099,0.003499,-0.105327,-0.154782,...,0.005787,-0.082780,-0.009048,0.046565,-0.040207,-0.089750,-0.103536,0.090986,-0.072441,0.074443,0.065187,-0.018859,0.063093,-0.122281,0.054790,0.119621,-0.117958,-0.008106,-0.071025,0.019424,-0.026795,-0.065976,-0.045075,-0.033450,0.004089,-0.091568,0.090431,0.037361,0.209537,-0.030037,-0.083185,-0.073892,-0.107537,0.053009,-0.203266,-0.116658,-0.052270,-0.067016,0.067302,non_Anti_microbial
2,0.111323,0.024840,0.122577,-0.200168,0.211283,-0.039343,-0.109409,0.027931,-0.040814,-0.111279,-0.036637,-0.010607,0.046858,0.062787,-0.046101,-0.027495,-0.069926,0.055665,0.011457,-0.077800,0.052512,0.011069,-0.041876,-0.195665,-0.018367,0.050151,0.093020,0.008231,0.008631,-0.097777,-0.039131,0.053937,-0.143924,0.058895,0.053202,-0.087114,0.087996,-0.057728,-0.081531,-0.049334,...,0.046643,-0.073014,-0.068133,0.090216,-0.050809,-0.036625,-0.086690,0.051850,0.001070,0.044721,-0.008755,-0.021717,0.047530,-0.103937,0.038788,0.063038,-0.065605,-0.035414,-0.062052,-0.000771,-0.018625,-0.052999,0.018849,-0.025234,-0.004726,-0.055147,0.102750,-0.021590,0.067670,-0.016135,0.022117,-0.088876,-0.142062,0.005176,-0.166395,-0.042158,-0.090205,-0.038277,0.002191,non_Anti_microbial


In [ ]:
data.value_counts("labels")

labels
non_Anti_microbial    9777
Anti_microbial        3268
dtype: int64

In [ ]:
labels = data["labels"]
data = data.drop("labels" , axis = 1)
print("Shape of data is :",data.shape,"Shape of labels is : ",labels.shape)

Shape of data is : (13045, 1024) Shape of labels is :  (13045,)


In [ ]:
from sklearn.model_selection import train_test_split
train_features,test_features,train_labels,test_labels = train_test_split(data, labels, test_size = 0.13, stratify=labels,random_state = 42)
print("Shape of train_features and train labels is :",train_features.shape,train_labels.shape)
print("Shape of test_features and test_labels is : ",test_features.shape,test_labels.shape)

Shape of train_features and train labels is : (11349, 1024) (11349,)
Shape of test_features and test_labels is :  (1696, 1024) (1696,)


In [ ]:
train_features["labels"] = list(train_labels)
test_features["labels"] = list(test_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
type(train_labels)

pandas.core.series.Series

In [ ]:
train_features.head(3)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,labels
2255,0.031776,-0.027921,-0.002877,-0.147194,0.150376,-0.001528,-0.064426,-0.047460,-0.005686,-0.110137,-0.098233,-0.021952,0.007496,-0.077937,0.001968,-0.015003,-0.041122,0.084479,0.057136,0.036204,0.057517,-0.022898,0.027018,-0.104637,-0.156715,-0.009038,0.041330,-0.003780,-0.039153,-0.076083,-0.095316,0.064067,-0.100659,-0.012934,0.073428,-0.048179,0.166126,-0.043250,-0.036603,0.006181,...,0.041017,-0.012835,0.016975,0.008138,-0.098891,-0.046668,-0.076522,0.082843,-0.029855,0.045958,-0.018832,-0.039814,0.107620,-0.057539,-0.061097,-0.027217,-0.023102,-0.026213,-0.083684,-0.040919,-0.025093,-0.058228,0.038443,-0.012139,0.006184,-0.035568,-0.008107,-0.082168,0.001727,-0.090185,-0.002516,-0.028781,-0.114518,-0.005395,-0.057053,-0.059022,-0.067482,-0.016724,0.002891,Anti_microbial
5716,0.135872,0.014606,0.076611,-0.295001,0.277320,-0.032924,-0.051026,-0.013421,-0.085678,-0.137909,0.020649,0.038876,0.039154,0.134754,0.066811,-0.152788,-0.058336,0.105534,-0.040341,-0.079608,0.068903,0.088370,-0.117220,-0.132376,0.034054,-0.009841,0.062385,0.030015,0.070556,-0.140085,0.008696,0.137880,-0.165311,0.010079,0.098351,-0.069681,0.043736,0.001666,-0.122446,-0.161463,...,-0.002398,-0.077538,0.011734,0.047796,-0.055281,-0.104313,-0.097418,0.086787,-0.093739,0.071810,0.065398,-0.006039,0.062626,-0.098479,0.061696,0.124932,-0.117388,0.021206,-0.067915,0.047174,-0.021232,-0.074441,-0.056596,-0.025545,0.005097,-0.097408,0.090363,0.062502,0.224509,-0.004505,-0.082303,-0.055964,-0.123756,0.053434,-0.203528,-0.116633,-0.043016,-0.081660,0.071516,non_Anti_microbial
6978,0.069585,0.009713,0.058913,-0.180797,0.195681,0.012911,-0.106612,0.057295,-0.020182,-0.081798,-0.060896,-0.046798,0.074334,0.090443,-0.009775,0.032702,-0.024203,0.062925,0.076867,-0.102591,0.040713,0.054539,-0.025717,-0.184562,-0.063081,-0.005208,0.044689,-0.034102,0.037094,-0.079286,-0.052217,0.011512,-0.106528,0.081739,0.071955,-0.131061,0.114489,-0.073059,-0.095921,-0.012154,...,0.006437,-0.110682,-0.099758,0.030435,-0.079064,-0.039414,-0.075532,-0.003134,-0.025852,0.076593,0.008769,-0.007426,0.031173,-0.051602,0.065502,0.094191,0.024978,-0.062388,-0.084072,-0.045510,-0.078724,-0.019195,0.061779,-0.001777,0.007160,-0.097733,0.089607,-0.030114,0.005077,-0.057750,-0.004976,-0.122645,-0.190197,-0.018957,-0.112847,-0.076487,-0.033950,-0.019898,0.021438,non_Anti_microbial


In [ ]:
test_features.head(3)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,labels
1690,0.109431,0.037502,0.059286,-0.212406,0.184761,-0.002387,-0.094443,-0.014094,-0.041196,-0.110593,0.023350,-0.002643,0.081095,0.111027,-0.002186,-0.043364,-0.025856,0.069302,0.017455,-0.037943,0.024812,0.029315,-0.106831,-0.113370,-0.063722,0.039092,0.029709,0.014165,0.056934,-0.089000,-0.013783,0.076565,-0.155654,0.089465,0.058875,-0.084667,0.042516,-0.025731,-0.108964,-0.149267,...,-0.013593,-0.052182,-0.058593,0.063475,0.032207,-0.052440,-0.028907,-0.010147,-0.077112,0.036872,0.025252,0.020158,0.035512,-0.116798,0.092976,0.078440,-0.007102,-0.060508,-0.082755,0.013825,-0.096522,-0.067320,0.025822,-0.029683,0.019120,-0.078460,0.108477,0.024281,0.147555,0.051941,-0.066832,-0.029229,-0.105289,0.083165,-0.097971,-0.097137,-0.023382,-0.123162,0.075368,non_Anti_microbial
5905,-0.008494,0.027247,-0.012789,-0.137007,0.175489,-0.046696,-0.078308,-0.009429,-0.004535,-0.101400,-0.076042,-0.028133,0.017943,0.001078,-0.001259,-0.000867,-0.027447,0.056188,0.022451,0.028725,0.021201,-0.026593,0.012847,-0.085048,-0.066358,-0.031304,0.017349,-0.054906,-0.043239,-0.134178,-0.078860,0.004786,-0.104230,0.038747,0.125431,-0.061640,0.153986,-0.027879,-0.036898,0.003091,...,0.029707,-0.019854,0.011796,-0.042385,-0.119466,-0.019402,-0.077659,0.093463,-0.006995,0.065490,-0.016651,-0.088461,0.095529,-0.033680,-0.010749,0.007390,-0.080337,-0.057983,-0.217866,-0.065110,-0.088622,-0.034897,0.011049,-0.003211,0.034794,-0.010049,-0.007819,-0.011402,0.064271,-0.049518,0.039848,-0.009495,-0.129781,-0.021887,-0.044729,-0.035315,-0.081538,-0.003337,-0.027138,Anti_microbial
4576,0.054417,0.094189,0.103464,-0.175528,0.163843,-0.040624,-0.144607,0.046408,-0.037799,-0.081449,-0.090296,-0.086181,0.072488,0.020609,-0.073911,0.134762,-0.121659,0.049699,0.090068,-0.019201,0.003705,0.051190,0.026391,-0.116375,-0.005507,-0.001336,0.099146,0.080520,-0.064099,-0.157723,-0.021806,-0.003806,-0.189323,-0.044014,0.040237,-0.001729,0.172977,-0.106001,0.066516,0.007342,...,0.032812,0.012422,-0.018214,-0.022703,-0.128591,0.010230,-0.117096,-0.056186,0.040505,0.032570,0.004417,-0.053004,0.028572,0.063831,0.051129,0.042063,-0.062931,-0.066595,-0.002475,-0.029575,-0.049054,-0.022394,0.027545,0.034095,0.038732,-0.018791,0.009082,-0.009632,-0.046492,-0.077531,-0.029261,-0.133262,-0.057388,-0.020919,-0.152536,-0.100700,-0.076835,0.096953,-0.035623,Anti_microbial


In [ ]:
feature = list(train_features.iloc[0])

In [ ]:
len(feature)

1024

In [ ]:
train_features.to_csv("train_dataframe.csv", encoding='utf-8', index=False)
print("Train_Dataframe Saved")

Train_Dataframe Saved


In [ ]:
test_features.to_csv("test_dataframe.csv", encoding='utf-8', index=False)
print("Test_Dataframe Saved")

Test_Dataframe Saved


In [ ]:
!cp "train_dataframe.csv" "/content/drive/MyDrive/Bayes labs"

**Modeling**

In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 925 kB 8.4 MB/s 
     |████████████████████████████████| 636 kB 60.2 MB/s 
     |████████████████████████████████| 829 kB 62.7 MB/s 
     |████████████████████████████████| 125 kB 48.3 MB/s 
     |████████████████████████████████| 282 kB 63.5 MB/s 
     |████████████████████████████████| 1.3 MB 62.2 MB/s 
     |████████████████████████████████| 294 kB 48.5 MB/s 
     |████████████████████████████████| 160 kB 69.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=0c3cc56a2f40d256af57d0a516bb0dc3d6834dfdd678dcc92c91705d63cfe9d1
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [ ]:
from torch import nn
from torch.nn.functional import relu 
import torch.nn.functional as F
from torch import tensor
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch
import pandas as pd
import numpy as np
#from pytorch_lightning.metrics.functional import accuracy
from torchmetrics.functional.classification.accuracy import accuracy

class TorchModel(nn.Module):
    def __init__(self, input_size=1024, hidden_size_1=64,hidden_size_2=128,
                 hidden_size_3=64,hidden_size_4=32,hidden_size_5= 32,
                 #hidden_size_5= 32,hidden_size_6 = 32,hidden_size_7 = 16,
                 output_size=2):
        super(TorchModel, self).__init__()
        self.layer_1 = torch.nn.Linear(input_size,hidden_size_1)
        self.layer_2 = torch.nn.Linear(hidden_size_1, hidden_size_2)
        self.layer_3 = torch.nn.Linear(hidden_size_2, hidden_size_3)
        self.layer_4 = torch.nn.Linear(hidden_size_3, hidden_size_4)
        self.layer_5 = torch.nn.Linear(hidden_size_4, hidden_size_5)
        #self.layer_6 = torch.nn.Linear(hidden_size_5, hidden_size_6)
        #self.layer_7 = torch.nn.Linear(hidden_size_6, hidden_size_7)
        self.output_layer = torch.nn.Linear(hidden_size_5, output_size)
       
    def forward(self,input_data):
        output_of_layer_1 = relu(self.layer_1(input_data))
        output_of_layer_2 = relu(self.layer_2(output_of_layer_1))
        output_of_layer_3 = relu(self.layer_3(output_of_layer_2))
        output_of_layer_4 = relu(self.layer_4(output_of_layer_3))
        output_of_layer_5 = relu(self.layer_5(output_of_layer_4))
        #output_of_layer_6 = relu(self.layer_6(output_of_layer_5))
        #output_of_layer_7 = relu(self.layer_7(output_of_layer_6))
        output = self.output_layer(output_of_layer_5)
        final_output = F.log_softmax(output,dim = 1)
        return final_output


class LightningModel(pl.LightningModule):
    def __init__(self):
        super(LightningModel, self).__init__()
        self.model = TorchModel()
        self.batch_size = 32
        #self.loss = nn.
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.001)

    def cross_entropy_loss(self, output, labels):
        return torch.nn.NLLLoss()(output, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        output = self.model.forward(x)
        train_loss = self.cross_entropy_loss(output, y)
        #predicted_labels = torch.argmax(output, dim=1)
        train_acc = accuracy(torch.argmax(output, dim=1), y)
        #pbar = {"train_acc":acc}
        self.log("train_performance", {"train_acc":train_acc, "train_loss": train_loss},on_step=False, on_epoch=True, prog_bar=True, logger=True)
        #self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return train_loss
    
    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        output = self.model.forward(x)
        test_loss = self.cross_entropy_loss(output, y)
        test_acc = accuracy(torch.argmax(output, dim=1), y)
        self.log("test_performance", {"test_acc":test_acc, "test_loss": test_loss},on_step=True, on_epoch=True, prog_bar=True, logger=True)
        #output = dict({'test_loss': test_loss,'test_acc': test_acc})


        return test_loss


    def prepare_data(self):
        train_data = pd.read_csv(r"/content/drive/MyDrive/Bayes labs/test_dataframe.csv")
        train_data["labels"] = train_data["labels"].map({"Anti_microbial":1 , "non_Anti_microbial" :0})
        train_labels = train_data["labels"]
        train_data = train_data.drop("labels" , axis = 1)
        train_data = torch.tensor(np.array(train_data), dtype=torch.float)
        train_labels=torch.LongTensor(np.array(train_labels))
        self.train_data = []
        for i in range(len(train_data)):
            self.train_data.append((train_data[i], train_labels[i]))
        # self.trn_data = (train_data, train_labels)


        test_data = pd.read_csv(r"/content/drive/MyDrive/Bayes labs/test_dataframe.csv")
        test_data["labels"] = test_data["labels"].map({"Anti_microbial":1 , "non_Anti_microbial" :0})
        test_labels = test_data["labels"]
        test_data = test_data.drop("labels" , axis = 1)
        test_data = (torch.tensor(np.array(test_data), dtype=torch.float))
        test_labels=torch.LongTensor(np.array(test_labels))
        self.test_data = []
        for i in range(len(train_data)):
            self.test_data.append((test_data[i], test_labels[i]))


    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size = self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size = self.batch_size)


trainer = pl.Trainer(max_epochs=150)
model = LightningModel()
trainer.fit(model)
trainer.test(model,dataloaders = model.test_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | TorchModel | 85.4 K
-------------------------------------
85.4 K    Trainable params
0         Non-trainable params
85.4 K    Total params
0.342     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_performance': {'test_acc': tensor(0.9788), 'test_loss': tensor(0.0746)},
 'test_performance_epoch': {'test_acc': tensor(0.9788),
                            'test_loss': tensor(0.0746)}}
--------------------------------------------------------------------------------


[{'test_performance': {'test_acc': tensor(0.9788),
   'test_loss': tensor(0.0746)},
  'test_performance_epoch': {'test_acc': tensor(0.9788),
   'test_loss': tensor(0.0746)}}]

In [ ]:
trainer.save_checkpoint("example.ckpt")


In [ ]:
new_model = MyModel.load_from_checkpoint(checkpoint_path="lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt")

In [ ]:
! ls lightning_logs/version_1/checkpoints/'epoch=119-step=3239.ckpt'


'lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt'


In [ ]:
trainer = Trainer(default_root_dir="lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt")


In [ ]:
new_model = MyModel.load_from_checkpoint(checkpoint_path="lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt")
